In [1]:
import pandas as pd
import numpy as np

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver

from bs4 import BeautifulSoup 
import requests 

import re
from tqdm import tnrange, tqdm_notebook as tqdm
import time
import random
from copy import deepcopy

import pickle

In [2]:
#start driver window
driver = webdriver.Chrome()

In [3]:
driver.get('https://pokemondb.net/pokedex/all')

In [4]:
#creates list of pokemon
pokedex = []

In [5]:
#searches each row
for i in driver.find_element_by_id('pokedex').find_elements_by_css_selector('tr'):
    pokedex.append(i.text)

In [6]:
pd.DataFrame(pokedex)

,0
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp....
1,001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45
2,002 Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60
3,003 Venusaur GRASS\nPOISON 525 80 82 83 100 10...
4,003 Venusaur\nMega Venusaur GRASS\nPOISON 625 ...
...,...
1024,888 Zacian\nHero of Many Battles FAIRY\n670 92...
1025,889 Zamazenta\nCrowned Shield FIGHTING\nSTEEL ...
1026,889 Zamazenta\nHero of Many Battles FIGHTING\n...
1027,890 Eternatus POISON\nDRAGON 690 140 85 95 145...


In [7]:
#Save in dataframe
df = pd.DataFrame(pokedex)

# Name

In [8]:
# Extracts names that start with a capital, then has smaller letter 1 or more 
# Can have a spce
df['name'] = df[0].str.extract(r'([A-Z][a-z.-]+\s?)')

In [9]:
df['name']

0            Name\n
1        Bulbasaur 
2          Ivysaur 
3         Venusaur 
4        Venusaur\n
           ...     
1024       Zacian\n
1025    Zamazenta\n
1026    Zamazenta\n
1027     Eternatus 
1028    Eternatus\n
Name: name, Length: 1029, dtype: object

# Secondary Name

In [10]:
# separating the real names from the rest of the column
df['temp_column'] = df[0].str.split('\n',expand=True)[1]

In [11]:
# extracts secondary names, uses same regex as above but we now need spaces
df['secondary_name']= df['temp_column'].str.extract(r'([A-Z1-9][a-z.-1-9%]+\s?[A-Za-z.-]+\s?[A-Za-z.-]+?\s)')

In [12]:
# fixing name column. Getting rid of the line breaks
# to separate the lines by the new line split
df['name'] = df['name'].str.split('\n',expand=True)[0]

In [13]:
df.head()

,0,name,temp_column,secondary_name
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp....,Name,Name,NaN
1,001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45,Bulbasaur,POISON 318 45 49 49 65 65 45,NaN
2,002 Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60,Ivysaur,POISON 405 60 62 63 80 80 60,NaN
3,003 Venusaur GRASS\nPOISON 525 80 82 83 100 10...,Venusaur,POISON 525 80 82 83 100 100 80,NaN
4,003 Venusaur\nMega Venusaur GRASS\nPOISON 625 ...,Venusaur,Mega Venusaur GRASS,Mega Venusaur


# Pokedex Numbers

In [14]:
#just get the first 3 numbers grouping in the line
df['pokedex_number'] = df[0].str.extract(r'([0-9]{3})')

In [16]:
df.head()

,0,name,temp_column,secondary_name,pokedex_number
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp....,Name,Name,NaN,NaN
1,001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45,Bulbasaur,POISON 318 45 49 49 65 65 45,NaN,001
2,002 Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60,Ivysaur,POISON 405 60 62 63 80 80 60,NaN,002
3,003 Venusaur GRASS\nPOISON 525 80 82 83 100 10...,Venusaur,POISON 525 80 82 83 100 100 80,NaN,003
4,003 Venusaur\nMega Venusaur GRASS\nPOISON 625 ...,Venusaur,Mega Venusaur GRASS,Mega Venusaur,003


# Types

In [17]:
# code to get all capitalized words (this would be the types)
df[0].str.findall(r'([A-Z][A-Z]+)')

0                    [HP]
1         [GRASS, POISON]
2         [GRASS, POISON]
3         [GRASS, POISON]
4         [GRASS, POISON]
              ...        
1024              [FAIRY]
1025    [FIGHTING, STEEL]
1026           [FIGHTING]
1027     [POISON, DRAGON]
1028     [POISON, DRAGON]
Name: 0, Length: 1029, dtype: object

In [18]:
df['type'] = df[0].str.findall(r'([A-Z][A-Z]+)')

In [19]:
df.head()

,0,name,temp_column,secondary_name,pokedex_number,type
0,#\nName\nType\nTotal\nHP\nAttack\nDefense\nSp....,Name,Name,NaN,NaN,[HP]
1,001 Bulbasaur GRASS\nPOISON 318 45 49 49 65 65 45,Bulbasaur,POISON 318 45 49 49 65 65 45,NaN,001,"[GRASS, POISON]"
2,002 Ivysaur GRASS\nPOISON 405 60 62 63 80 80 60,Ivysaur,POISON 405 60 62 63 80 80 60,NaN,002,"[GRASS, POISON]"
3,003 Venusaur GRASS\nPOISON 525 80 82 83 100 10...,Venusaur,POISON 525 80 82 83 100 100 80,NaN,003,"[GRASS, POISON]"
4,003 Venusaur\nMega Venusaur GRASS\nPOISON 625 ...,Venusaur,Mega Venusaur GRASS,Mega Venusaur,003,"[GRASS, POISON]"


# Powers

In [21]:
# we need to get all the numbers, but do not want to include the pokedex numbers
df['test']= df[0].str.split(' ',1,expand=True)[1]

In [23]:
#takes all the numbers from single digits to four digits
test = df['test'].str.findall(r'([1-9][0-9]?[0-9]?[0-9]?)')

In [24]:
# turn test dataframe  into list
#turn into dataframe
powers = pd.DataFrame(test.to_list())

In [29]:
# column 7 should not be there 
# find out why: 3 pokemon with numbers in their name
df[~powers[7].isna()]

,0,name,temp_column,secondary_name,pokedex_number,type,test
280,233 Porygon2 NORMAL\n515 85 80 90 105 95 60,Porygon,515 85 80 90 105 95 60,NaN,233,[NORMAL],Porygon2 NORMAL\n515 85 80 90 105 95 60
834,718 Zygarde\n50% Forme DRAGON\nGROUND 600 108 ...,Zygarde,50% Forme DRAGON,50% Forme,718,"[DRAGON, GROUND]",Zygarde\n50% Forme DRAGON\nGROUND 600 108 100 ...
835,718 Zygarde\n10% Forme DRAGON\nGROUND 486 54 1...,Zygarde,10% Forme DRAGON,10% Forme,718,"[DRAGON, GROUND]",Zygarde\n10% Forme DRAGON\nGROUND 486 54 100 7...


In [30]:
#fix these manually on the test list before turning into dataframe
test[280].remove('2')

test[834].remove('50')

test[835].remove('10')

In [31]:
# save into df
powers = pd.DataFrame(test.to_list())

In [33]:
# rename for ease
powers.rename(columns = {0:'total',1:'hp',2:'attack',3:'defense',4:'sp_attack',5:'sp_defense',6:'speed'}, inplace=True)

In [34]:
powers.head()

,total,hp,attack,defense,sp_attack,sp_defense,speed
0,None,None,None,None,None,None,None
1,318,45,49,49,65,65,45
2,405,60,62,63,80,80,60
3,525,80,82,83,100,100,80
4,625,80,100,123,122,120,80


In [35]:
# merging based on indexes
df = df.merge(powers, how='inner', left_index=True, right_index=True)

# Final Cleanup

In [37]:
#drop columns that aren't needed
df.drop(columns = [0, 'temp_column','test'], inplace=True)

In [38]:
#drop 1st row
df.drop(0,inplace=True)

In [39]:
df

,name,secondary_name,pokedex_number,type,total,hp,attack,defense,sp_attack,sp_defense,speed
1,Bulbasaur,NaN,001,"[GRASS, POISON]",318,45,49,49,65,65,45
2,Ivysaur,NaN,002,"[GRASS, POISON]",405,60,62,63,80,80,60
3,Venusaur,NaN,003,"[GRASS, POISON]",525,80,82,83,100,100,80
4,Venusaur,Mega Venusaur,003,"[GRASS, POISON]",625,80,100,123,122,120,80
5,Charmander,NaN,004,[FIRE],309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...
1024,Zacian,Hero of Many,888,[FAIRY],670,92,130,115,80,115,138
1025,Zamazenta,Crowned Shield,889,"[FIGHTING, STEEL]",720,92,130,145,80,145,128
1026,Zamazenta,Hero of Many,889,[FIGHTING],670,92,130,115,80,115,138
1027,Eternatus,NaN,890,"[POISON, DRAGON]",690,140,85,95,145,95,130


# Merge with vw data

In [40]:
ls

README.md                         python_screener_2-practice.ipynb
model.csv                         python_screener_2.ipynb
model_props.csv                   python_screener_2_working.ipynb
partner.csv                       python_screener_blank.html
pokemon_df.pkl                    scrape_pokemon.ipynb
pokemon_df1.pkl                   vestwell_partner_db.csv
pokemon_df_full.pkl


In [104]:
vw = pd.read_csv('model_props.csv')

In [68]:
vw_pokemon = vw.symbol.to_list()

In [69]:
vw_pokemon = list(set(vw_pokemon))

In [52]:
# match names with vestwell
df['combined_name'] = df['name']+df['secondary_name']

In [55]:
# fill empty names with just first names
df['combined_name'].fillna(df['name'], inplace=True)

In [64]:
#strips spaces from the beginning and end of string
df['combined_name'] = df['combined_name'].str.strip()

In [70]:
check_list = df['combined_name'].to_list()

In [73]:
for i in vw_pokemon:
    if i not in check_list:
        print(i)

Mr. Mime
Farfetch'd
Porygon2


In [ ]:
# Farfetch on the scrape df
# Mr. on the scrape df
# Porygon on the scrape df. Must fix in REGEX

In [83]:
df[df['combined_name'] == 'Mr.']

,name,secondary_name,pokedex_number,type,total,hp,attack,defense,sp_attack,sp_defense,speed,combined_name
156,Mr.,NaN,122,"[PSYCHIC, FAIRY]",460,40,45,65,100,120,90,Mr.
998,Mr.,NaN,866,"[PSYCHIC, ICE]",520,80,85,75,110,100,70,Mr.


In [84]:
df['combined_name'].replace('Mr.','Mr. Mime', inplace=True)

In [85]:
df[df['combined_name'] == 'Mr.']

,name,secondary_name,pokedex_number,type,total,hp,attack,defense,sp_attack,sp_defense,speed,combined_name


In [88]:
df['combined_name'].replace('Farfetch',"Farfetch'd", inplace=True)

In [89]:
df[df['combined_name'] == 'Farfetch']

,name,secondary_name,pokedex_number,type,total,hp,attack,defense,sp_attack,sp_defense,speed,combined_name


In [90]:
df[df['combined_name'] == "Farfetch'd"] 

,name,secondary_name,pokedex_number,type,total,hp,attack,defense,sp_attack,sp_defense,speed,combined_name
109,Farfetch,NaN,083,"[NORMAL, FLYING]",377,52,90,55,58,62,60,Farfetch'd


In [91]:
check_list2 = df['combined_name'].to_list()

In [92]:
for i in vw_pokemon:
    if i not in check_list2:
        print(i)

Porygon2


In [93]:
df

,name,secondary_name,pokedex_number,type,total,hp,attack,defense,sp_attack,sp_defense,speed,combined_name
1,Bulbasaur,NaN,001,"[GRASS, POISON]",318,45,49,49,65,65,45,Bulbasaur
2,Ivysaur,NaN,002,"[GRASS, POISON]",405,60,62,63,80,80,60,Ivysaur
3,Venusaur,NaN,003,"[GRASS, POISON]",525,80,82,83,100,100,80,Venusaur
4,Venusaur,Mega Venusaur,003,"[GRASS, POISON]",625,80,100,123,122,120,80,VenusaurMega Venusaur
5,Charmander,NaN,004,[FIRE],309,39,52,43,60,50,65,Charmander
...,...,...,...,...,...,...,...,...,...,...,...,...
1024,Zacian,Hero of Many,888,[FAIRY],670,92,130,115,80,115,138,ZacianHero of Many
1025,Zamazenta,Crowned Shield,889,"[FIGHTING, STEEL]",720,92,130,145,80,145,128,ZamazentaCrowned Shield
1026,Zamazenta,Hero of Many,889,[FIGHTING],670,92,130,115,80,115,138,ZamazentaHero of Many
1027,Eternatus,NaN,890,"[POISON, DRAGON]",690,140,85,95,145,95,130,Eternatus


In [97]:
vw[vw['symbol']=='Porygon2']

,model_props_id,model_id,symbol
473,1854,261,Porygon2
528,1839,259,Porygon2
646,2742,355,Porygon2
658,2756,356,Porygon2


In [102]:
vw = vw.merge(df, how = 'left', left_on='symbol', right_on = 'combined_name')

In [105]:
vw

,model_props_id,model_id,symbol
0,541,80,Bulbasaur
1,542,80,Ivysaur
2,543,80,Venusaur
3,544,80,VenusaurMega Venusaur
4,545,80,Charmander
...,...,...,...
724,1841,260,Diglett
725,29155,260,Torchic
726,1842,260,Psyduck
727,1843,260,Paras


In [106]:
# extra two rows are because porygon has duplicate values. Merge keeps them both
vw.merge(df, how = 'left', left_on='symbol', right_on = 'combined_name')

,model_props_id,model_id,symbol,name,secondary_name,pokedex_number,type,total,hp,attack,defense,sp_attack,sp_defense,speed,combined_name
0,541,80,Bulbasaur,Bulbasaur,NaN,001,"[GRASS, POISON]",318,45,49,49,65,65,45,Bulbasaur
1,542,80,Ivysaur,Ivysaur,NaN,002,"[GRASS, POISON]",405,60,62,63,80,80,60,Ivysaur
2,543,80,Venusaur,Venusaur,NaN,003,"[GRASS, POISON]",525,80,82,83,100,100,80,Venusaur
3,544,80,VenusaurMega Venusaur,Venusaur,Mega Venusaur,003,"[GRASS, POISON]",625,80,100,123,122,120,80,VenusaurMega Venusaur
4,545,80,Charmander,Charmander,NaN,004,[FIRE],309,39,52,43,60,50,65,Charmander
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,1841,260,Diglett,Diglett,NaN,050,[GROUND],265,10,55,25,35,45,95,Diglett
727,29155,260,Torchic,Torchic,NaN,255,[FIRE],310,45,60,40,70,50,45,Torchic
728,1842,260,Psyduck,Psyduck,NaN,054,[WATER],320,50,52,48,65,50,55,Psyduck
729,1843,260,Paras,Paras,NaN,046,"[BUG, GRASS]",285,35,70,55,45,55,25,Paras


In [108]:
#saves it into a smaller df file
vw.to_pickle('merged_vestwell_pokemon.pkl')